In [1]:
import os
import gzip
import shutil
import tensorflow as tf
import six.moves.urllib_request as urllib
tf.enable_eager_execution()
from sklearn.model_selection import train_test_split

/Users/liuxinxin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data url
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"

In [3]:
# google download code
# if downloaded , done else download

def download_report_hook(count, block_size, total_size):
  """Report hook for download progress.

  Args:
    count: current block number
    block_size: block size
    total_size: total size
  """
  percent = int(count * block_size * 100 / total_size)
  print("\r%d%%" % percent + " completed", end="\r")


def maybe_download(directory, filename, uri):
  """Download filename from uri unless it's already in directory.

  Copies a remote file to local if that local file does not already exist.  If
  the local file pre-exists this function call, it does not check that the local
  file is a copy of the remote.

  Remote filenames can be filepaths, any URI readable by tensorflow.gfile, or a
  URL.

  Args:
    directory: path to the directory that will be used.
    filename: name of the file to download to (do nothing if it already exists).
    uri: URI to copy (or download) from.

  Returns:
    The path to the downloaded file.
  """
  tf.gfile.MakeDirs(directory)
  filepath = os.path.join(directory, filename)
  if tf.gfile.Exists(filepath):
    tf.logging.info("Not downloading, file already found: %s" % filepath)
    return filepath

  tf.logging.info("Downloading %s to %s" % (uri, filepath))
  try:
    tf.gfile.Copy(uri, filepath)
  except tf.errors.UnimplementedError:
    if uri.startswith("http"):
      inprogress_filepath = filepath + ".incomplete"
      inprogress_filepath, _ = urllib.urlretrieve(
          uri, inprogress_filepath, reporthook=download_report_hook)
      # Print newline to clear the carriage return from the download progress
      print()
      tf.gfile.Rename(inprogress_filepath, filepath)
    else:
      raise ValueError("Unrecognized URI: " + filepath)
  statinfo = os.stat(filepath)
  tf.logging.info("Successfully downloaded %s, %s bytes." %
                  (filename, statinfo.st_size))
  return filepath

In [4]:
# download the file to the data documents
# now you can get the covtype.data.gz file in your document
file_name = url.split('/')[-1]
maybe_download("",file_name, url)

INFO:tensorflow:Not downloading, file already found: covtype.data.gz


'covtype.data.gz'

In [5]:
# extract the data
# now you get covtype.data file
new_file_name = file_name.replace(".gz", "")
with gzip.open(file_name, 'rb') as f_in:
    with open(new_file_name, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [6]:
# devide train/test
# you get covtype.csv.train and covtype.csv.test file
data = []
with open(new_file_name,'r') as f:
    data = f.readlines()
train,test= train_test_split(data,test_size=0.1)

def save_file(name,dataset):
    with open(name,'w') as f:
        for data in dataset:
            f.write(data)

NUM_TRAIN_EXAMPLES = len(train)

save_file('covtype.csv.train',train)
save_file('covtype.csv.test',test)

In [7]:
# we can forward to the tutorail
# maybe google can print more beatiful
defaults = [tf.int32] * 55
dataset = tf.contrib.data.CsvDataset(['covtype.csv.train'],defaults)
print(list(dataset.take(1)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.CsvDataset(...)`.
Instructions for updating:
Colocations handled automatically by placer.
[(<tf.Tensor: id=71, shape=(), dtype=int32, numpy=2866>, <tf.Tensor: id=72, shape=(), dtype=int32, numpy=15>, <tf.Tensor: id=73, shape=(), dtype=int32, numpy=24>, <tf.Tensor: id=74, shape=(), dtype=int32, numpy=384>, <tf.Tensor: id=75, shape=(), dtype=int32, numpy=61>, <tf.Tensor: id=76, shape=(), dtype=int32, numpy=2658>, <tf.Tensor: id=77, shape=(), dtype=int32, numpy=192>, <tf.Tensor: id=78, shape=(), dtype=int32, numpy=183>, <tf.Tensor: id=79, shape=(), dtype=int32, numpy=120>, <tf.Tensor: id=80, shape=(), dtype=int32, numpy=1667>, <tf.Tensor: id=81, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=82, shape=()

In [8]:
# col names 
# come from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info spend most of my time :/
col_names = [
    'elevation',
    'aspect',
    'slope',
    'horizontal_distance_to_hydrology',
    'vertical_distance_to_hydrology',
    'horizontal_distance_to_roadways',
    'hillshade_9am',
    'hillshade_noon',
    'hillshade_3pm',
    'horizontal_distance_to_fire_points',
    'soil_type',
    'cover_type'
]

In [9]:
# parsing every line
def _parse_csv_row(*vals):
    soil_type_t = tf.convert_to_tensor(vals[14:54])
    feat_vals = vals[:10] + (soil_type_t,vals[54])
    features = dict(zip(col_names,feat_vals))
    
    class_label = tf.argmax(vals[10:14],axis=0)
    return features,class_label

In [10]:
# apply the parse to the dataset
# Until here is part 1
dataset = dataset.map(_parse_csv_row).batch(1)
print(list(dataset.take(1)))

[({'elevation': <tf.Tensor: id=255, shape=(1,), dtype=int32, numpy=array([2866], dtype=int32)>, 'aspect': <tf.Tensor: id=253, shape=(1,), dtype=int32, numpy=array([15], dtype=int32)>, 'slope': <tf.Tensor: id=262, shape=(1,), dtype=int32, numpy=array([24], dtype=int32)>, 'horizontal_distance_to_hydrology': <tf.Tensor: id=260, shape=(1,), dtype=int32, numpy=array([384], dtype=int32)>, 'vertical_distance_to_hydrology': <tf.Tensor: id=264, shape=(1,), dtype=int32, numpy=array([61], dtype=int32)>, 'horizontal_distance_to_roadways': <tf.Tensor: id=261, shape=(1,), dtype=int32, numpy=array([2658], dtype=int32)>, 'hillshade_9am': <tf.Tensor: id=257, shape=(1,), dtype=int32, numpy=array([192], dtype=int32)>, 'hillshade_noon': <tf.Tensor: id=258, shape=(1,), dtype=int32, numpy=array([183], dtype=int32)>, 'hillshade_3pm': <tf.Tensor: id=256, shape=(1,), dtype=int32, numpy=array([120], dtype=int32)>, 'horizontal_distance_to_fire_points': <tf.Tensor: id=259, shape=(1,), dtype=int32, numpy=array([16

In [11]:
# defining features
# Cover_Type / interger / 1 to 7
# until I type down this code , tensorflow not support the densefeature API
cover_type = tf.feature_column.categorical_column_with_identity('cover_type',num_buckets=8)
cover_embedding = tf.feature_column.embedding_column(cover_type,dimension=10)

numeric_cols = [
    'elevation',
    'aspect',
    'slope',
    'horizontal_distance_to_hydrology',
    'vertical_distance_to_hydrology',
    'horizontal_distance_to_roadways',
    'hillshade_9am',
    'hillshade_noon',
    'hillshade_3pm',
    'horizontal_distance_to_fire_points',
]

numeric_features = [tf.feature_column.numeric_column(feat) for feat in numeric_cols]

# Soil_Type (40 binary columns)
soil_type = tf.feature_column.numeric_column('soil_type',shape=(40,))

columns = numeric_features + [soil_type,cover_embedding]


In [12]:
# load data pipeline 
def load_data(*filenames):
    dataset =tf.contrib.data.CsvDataset(
        filenames,defaults
    )
    dataset = dataset.map(_parse_csv_row)
    dataset = dataset.batch(64)
    
    return dataset

In [13]:
# let us try esimator
model = tf.estimator.DNNLinearCombinedClassifier(
    linear_feature_columns = [cover_type,soil_type],
    dnn_feature_columns = numeric_features,
    dnn_hidden_units=[256,16,8],
    n_classes = 4    
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/78/tp45tbbn6pn1yvhmk_wnzgvc0000gn/T/tmp3fgkw_es', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1371e77f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# train the estimator
for epoch in range(10):
    # train 10
    model.train(input_fn=lambda:load_data('covtype.csv.train'))
    # evaluate every epoch
    print('epoch {}'.format(str(epoch + 1)))
    print(model.evaluate(input_fn=lambda:load_data('covtype.csv.test')))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was 

INFO:tensorflow:global_step/sec: 141.614
INFO:tensorflow:loss = 106.62742, step = 5401 (0.706 sec)
INFO:tensorflow:global_step/sec: 141.963
INFO:tensorflow:loss = 75.11008, step = 5501 (0.704 sec)
INFO:tensorflow:global_step/sec: 142.164
INFO:tensorflow:loss = 75.63512, step = 5601 (0.703 sec)
INFO:tensorflow:global_step/sec: 141.759
INFO:tensorflow:loss = 64.55018, step = 5701 (0.705 sec)
INFO:tensorflow:global_step/sec: 141.977
INFO:tensorflow:loss = 91.96159, step = 5801 (0.704 sec)
INFO:tensorflow:global_step/sec: 142.124
INFO:tensorflow:loss = 47.766426, step = 5901 (0.704 sec)
INFO:tensorflow:global_step/sec: 141.023
INFO:tensorflow:loss = 62.499176, step = 6001 (0.709 sec)
INFO:tensorflow:global_step/sec: 139.324
INFO:tensorflow:loss = 108.46098, step = 6101 (0.718 sec)
INFO:tensorflow:global_step/sec: 131.981
INFO:tensorflow:loss = 47.840134, step = 6201 (0.758 sec)
INFO:tensorflow:global_step/sec: 138.485
INFO:tensorflow:loss = 82.74372, step = 6301 (0.722 sec)
INFO:tensorflow

INFO:tensorflow:loss = 62.020393, step = 11572 (0.918 sec)
INFO:tensorflow:global_step/sec: 112.062
INFO:tensorflow:loss = 57.938293, step = 11672 (0.893 sec)
INFO:tensorflow:global_step/sec: 107.129
INFO:tensorflow:loss = 50.183178, step = 11772 (0.933 sec)
INFO:tensorflow:global_step/sec: 110.358
INFO:tensorflow:loss = 54.108467, step = 11872 (0.906 sec)
INFO:tensorflow:global_step/sec: 137.774
INFO:tensorflow:loss = 45.41421, step = 11972 (0.726 sec)
INFO:tensorflow:global_step/sec: 134.073
INFO:tensorflow:loss = 63.76265, step = 12072 (0.746 sec)
INFO:tensorflow:global_step/sec: 140.076
INFO:tensorflow:loss = 70.94657, step = 12172 (0.714 sec)
INFO:tensorflow:global_step/sec: 138.715
INFO:tensorflow:loss = 69.443054, step = 12272 (0.721 sec)
INFO:tensorflow:global_step/sec: 139.248
INFO:tensorflow:loss = 61.858894, step = 12372 (0.718 sec)
INFO:tensorflow:global_step/sec: 140.589
INFO:tensorflow:loss = 40.137974, step = 12472 (0.711 sec)
INFO:tensorflow:global_step/sec: 136.896
INF

INFO:tensorflow:global_step/sec: 143.005
INFO:tensorflow:loss = 30.119656, step = 18443 (0.699 sec)
INFO:tensorflow:global_step/sec: 141.857
INFO:tensorflow:loss = 42.288445, step = 18543 (0.705 sec)
INFO:tensorflow:global_step/sec: 142.406
INFO:tensorflow:loss = 62.02924, step = 18643 (0.702 sec)
INFO:tensorflow:global_step/sec: 143.402
INFO:tensorflow:loss = 45.87024, step = 18743 (0.697 sec)
INFO:tensorflow:global_step/sec: 142.926
INFO:tensorflow:loss = 59.162064, step = 18843 (0.700 sec)
INFO:tensorflow:global_step/sec: 142.638
INFO:tensorflow:loss = 34.310535, step = 18943 (0.701 sec)
INFO:tensorflow:global_step/sec: 142.622
INFO:tensorflow:loss = 53.052925, step = 19043 (0.701 sec)
INFO:tensorflow:global_step/sec: 142.843
INFO:tensorflow:loss = 58.948036, step = 19143 (0.700 sec)
INFO:tensorflow:global_step/sec: 143.287
INFO:tensorflow:loss = 35.003166, step = 19243 (0.698 sec)
INFO:tensorflow:global_step/sec: 142.444
INFO:tensorflow:loss = 51.928593, step = 19343 (0.702 sec)
IN

INFO:tensorflow:loss = 39.37002, step = 25214 (0.710 sec)
INFO:tensorflow:global_step/sec: 138.518
INFO:tensorflow:loss = 29.328348, step = 25314 (0.722 sec)
INFO:tensorflow:global_step/sec: 137.473
INFO:tensorflow:loss = 32.678146, step = 25414 (0.727 sec)
INFO:tensorflow:global_step/sec: 139.317
INFO:tensorflow:loss = 41.66727, step = 25514 (0.718 sec)
INFO:tensorflow:global_step/sec: 123.753
INFO:tensorflow:loss = 46.52503, step = 25614 (0.808 sec)
INFO:tensorflow:global_step/sec: 137.187
INFO:tensorflow:loss = 29.211008, step = 25714 (0.729 sec)
INFO:tensorflow:global_step/sec: 111.71
INFO:tensorflow:loss = 34.726696, step = 25814 (0.895 sec)
INFO:tensorflow:global_step/sec: 135.345
INFO:tensorflow:loss = 48.062252, step = 25914 (0.739 sec)
INFO:tensorflow:global_step/sec: 116.553
INFO:tensorflow:loss = 23.377094, step = 26014 (0.858 sec)
INFO:tensorflow:global_step/sec: 136.659
INFO:tensorflow:loss = 38.81402, step = 26114 (0.732 sec)
INFO:tensorflow:global_step/sec: 139.465
INFO:

...

INFO:tensorflow:global_step/sec: 138.768
INFO:tensorflow:loss = 32.038303, step = 53727 (0.721 sec)
INFO:tensorflow:global_step/sec: 135.613
INFO:tensorflow:loss = 31.493565, step = 53827 (0.737 sec)
INFO:tensorflow:global_step/sec: 135.88
INFO:tensorflow:loss = 22.664001, step = 53927 (0.736 sec)
INFO:tensorflow:global_step/sec: 133.747
INFO:tensorflow:loss = 30.924688, step = 54027 (0.747 sec)
INFO:tensorflow:global_step/sec: 132.018
INFO:tensorflow:loss = 29.00423, step = 54127 (0.757 sec)
INFO:tensorflow:global_step/sec: 134.395
INFO:tensorflow:loss = 21.88012, step = 54227 (0.744 sec)
INFO:tensorflow:global_step/sec: 136.35
INFO:tensorflow:loss = 17.13918, step = 54327 (0.733 sec)
INFO:tensorflow:global_step/sec: 137.199
INFO:tensorflow:loss = 23.089523, step = 54427 (0.729 sec)
INFO:tensorflow:global_step/sec: 135.297
INFO:tensorflow:loss = 27.222649, step = 54527 (0.739 sec)
INFO:tensorflow:global_step/sec: 135.102
INFO:tensorflow:loss = 39.416485, step = 54627 (0.740 sec)
INFO:

INFO:tensorflow:global_step/sec: 136.446
INFO:tensorflow:loss = 45.548134, step = 60498 (0.733 sec)
INFO:tensorflow:global_step/sec: 139.421
INFO:tensorflow:loss = 37.18619, step = 60598 (0.717 sec)
INFO:tensorflow:global_step/sec: 136.645
INFO:tensorflow:loss = 29.621431, step = 60698 (0.732 sec)
INFO:tensorflow:global_step/sec: 133.9
INFO:tensorflow:loss = 21.997341, step = 60798 (0.747 sec)
INFO:tensorflow:global_step/sec: 140.005
INFO:tensorflow:loss = 25.411812, step = 60898 (0.714 sec)
INFO:tensorflow:global_step/sec: 134.069
INFO:tensorflow:loss = 25.779577, step = 60998 (0.746 sec)
INFO:tensorflow:global_step/sec: 116.733
INFO:tensorflow:loss = 31.322132, step = 61098 (0.856 sec)
INFO:tensorflow:global_step/sec: 136.875
INFO:tensorflow:loss = 30.118866, step = 61198 (0.731 sec)
INFO:tensorflow:global_step/sec: 138.714
INFO:tensorflow:loss = 22.862434, step = 61298 (0.721 sec)
INFO:tensorflow:global_step/sec: 137.485
INFO:tensorflow:loss = 32.721733, step = 61398 (0.728 sec)
INF

INFO:tensorflow:global_step/sec: 136.867
INFO:tensorflow:loss = 23.675339, step = 67269 (0.731 sec)
INFO:tensorflow:global_step/sec: 139.256
INFO:tensorflow:loss = 16.074688, step = 67369 (0.718 sec)
INFO:tensorflow:global_step/sec: 135.39
INFO:tensorflow:loss = 12.749119, step = 67469 (0.739 sec)
INFO:tensorflow:global_step/sec: 135.207
INFO:tensorflow:loss = 26.492455, step = 67569 (0.739 sec)
INFO:tensorflow:global_step/sec: 140.664
INFO:tensorflow:loss = 30.022661, step = 67669 (0.711 sec)
INFO:tensorflow:global_step/sec: 140.35
INFO:tensorflow:loss = 18.172443, step = 67769 (0.713 sec)
INFO:tensorflow:global_step/sec: 137.362
INFO:tensorflow:loss = 18.240063, step = 67869 (0.728 sec)
INFO:tensorflow:global_step/sec: 137.445
INFO:tensorflow:loss = 12.078921, step = 67969 (0.728 sec)
INFO:tensorflow:global_step/sec: 137.855
INFO:tensorflow:loss = 26.56986, step = 68069 (0.726 sec)
INFO:tensorflow:global_step/sec: 138.349
INFO:tensorflow:loss = 28.997692, step = 68169 (0.723 sec)
INF

INFO:tensorflow:global_step/sec: 139.468
INFO:tensorflow:loss = 32.658215, step = 74040 (0.717 sec)
INFO:tensorflow:global_step/sec: 140.359
INFO:tensorflow:loss = 41.649605, step = 74140 (0.712 sec)
INFO:tensorflow:global_step/sec: 139.431
INFO:tensorflow:loss = 21.851551, step = 74240 (0.717 sec)
INFO:tensorflow:global_step/sec: 140.365
INFO:tensorflow:loss = 16.407444, step = 74340 (0.712 sec)
INFO:tensorflow:global_step/sec: 140.332
INFO:tensorflow:loss = 17.248558, step = 74440 (0.713 sec)
INFO:tensorflow:global_step/sec: 139.099
INFO:tensorflow:loss = 27.114298, step = 74540 (0.719 sec)
INFO:tensorflow:global_step/sec: 140.204
INFO:tensorflow:loss = 27.0438, step = 74640 (0.713 sec)
INFO:tensorflow:global_step/sec: 140.708
INFO:tensorflow:loss = 20.8899, step = 74740 (0.711 sec)
INFO:tensorflow:global_step/sec: 137.844
INFO:tensorflow:loss = 19.319885, step = 74840 (0.726 sec)
INFO:tensorflow:global_step/sec: 139.49
INFO:tensorflow:loss = 31.620684, step = 74940 (0.717 sec)
INFO:

INFO:tensorflow:Finished evaluation at 2019-01-10-08:28:37
INFO:tensorflow:Saving dict for global step 81710: accuracy = 0.86843824, average_loss = 0.3358376, global_step = 81710, loss = 21.489908
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 81710: /var/folders/78/tp45tbbn6pn1yvhmk_wnzgvc0000gn/T/tmp3fgkw_es/model.ckpt-81710
{'accuracy': 0.86843824, 'average_loss': 0.3358376, 'loss': 21.489908, 'global_step': 81710}


In [15]:
# generate input recevier funciont
features_sample =list(dataset.take(1))[0][0]
input_recevier_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(features_sample)

In [16]:
# export model
model.export_saved_model(
    export_dir_base = 'wide_deep',
    serving_input_receiver_fn = input_recevier_fn
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'elevation': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int32>, 'aspect': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int32>, 'slope': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int32>, 'horizon

b'wide_deep/1547109016'

In [ ]:
# Because of the dense features cant work, the keras model part cant work
# get the feature layer
feature_layer = tf.feature_column.DenseFeatures(columns)
# Building a model
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(8),    
    tf.keras.layers.Dense(4,activation=tf.nn.softmax)
])

# compile the model
model.compile(
    optimizer = tf.train.AdadeltaOptimizer(),
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# train the data
model.fit(dataset,steps_per_epoch=2000)

# export model
export_dir = tf.contrib.saved_model.save_keras_model(model,'keras_nn')
print(export_dir)

# restore model
restore_model = tf.contrib.saved_model.load_keras_model(export_dir)